# Parallelism

In [22]:
import numpy as np

import jax
import jax.numpy as jnp

from jax import jit, random, pmap, vmap, lax

### Numpy baseline

In [2]:
x = np.ones((8, 4096, 4096))
np.matmul(x, x).shape

(8, 4096, 4096)

In [3]:
%timeit -n 5 -r 5 np.matmul(x, x)

1.78 s ± 35.9 ms per loop (mean ± std. dev. of 5 runs, 5 loops each)


### Jax acceleration

In [4]:
jax.devices()

[TpuDevice(id=0, process_index=0, coords=(0,0,0), core_on_chip=0),
 TpuDevice(id=1, process_index=0, coords=(0,0,0), core_on_chip=1),
 TpuDevice(id=2, process_index=0, coords=(1,0,0), core_on_chip=0),
 TpuDevice(id=3, process_index=0, coords=(1,0,0), core_on_chip=1),
 TpuDevice(id=4, process_index=0, coords=(0,1,0), core_on_chip=0),
 TpuDevice(id=5, process_index=0, coords=(0,1,0), core_on_chip=1),
 TpuDevice(id=6, process_index=0, coords=(1,1,0), core_on_chip=0),
 TpuDevice(id=7, process_index=0, coords=(1,1,0), core_on_chip=1)]

In [5]:
x = jnp.ones((8, 4096, 4096))
jnp.matmul(x, x).shape

(8, 4096, 4096)

In [6]:
%timeit -n 5 -r 5 jnp.matmul(x, x).block_until_ready()

20.8 ms ± 102 µs per loop (mean ± std. dev. of 5 runs, 5 loops each)


### PMAP

In [7]:

xd = pmap(lambda x: jnp.ones((4096, 4096)))(np.arange(8))
print(xd.shape)
print(type(xd))

(8, 4096, 4096)
<class 'jaxlib.xla_extension.pmap_lib.ShardedDeviceArray'>


In [10]:
yd = pmap(jnp.matmul)(xd, xd)
print(yd.shape)
print(type(yd))

(8, 4096, 4096)
<class 'jaxlib.xla_extension.pmap_lib.ShardedDeviceArray'>


In [11]:
yd[0]

DeviceArray([[4096., 4096., 4096., ..., 4096., 4096., 4096.],
             [4096., 4096., 4096., ..., 4096., 4096., 4096.],
             [4096., 4096., 4096., ..., 4096., 4096., 4096.],
             ...,
             [4096., 4096., 4096., ..., 4096., 4096., 4096.],
             [4096., 4096., 4096., ..., 4096., 4096., 4096.],
             [4096., 4096., 4096., ..., 4096., 4096., 4096.]],            dtype=float32)

In [20]:
%timeit -r 5 -n 5 pmap(jnp.matmul)(xd, xd).block_until_ready()

4.45 ms ± 357 µs per loop (mean ± std. dev. of 5 runs, 5 loops each)


### Collective operations 

In [38]:
def normalize(x):
    return x / lax.psum(x, axis_name='p')

y_n = pmap(normalize, axis_name='p')(jnp.arange(8))
print(y_n.shape)
print(y_n)
print(y_n.sum())

(8,)
[0.         0.03571429 0.07142857 0.10714287 0.14285715 0.17857143
 0.21428573 0.25      ]
1.0


In [41]:
x = np.ones(8)
x

array([1., 1., 1., 1., 1., 1., 1., 1.])

In [42]:
pmap(lambda x: jax.lax.psum(x, axis_name='i'), axis_name='i')(x)

ShardedDeviceArray([8., 8., 8., 8., 8., 8., 8., 8.], dtype=float32)

### PJIT

In [43]:
import jax
from jax.experimental import maps
from jax.experimental import PartitionSpec
from jax.experimental.pjit import pjit
import numpy as np

In [44]:
mesh_shape = (4, 2)
devices = np.asarray(jax.devices()).reshape(*mesh_shape)
# 'x', 'y' axis names are used here for simplicity
mesh = maps.Mesh(devices, ('x', 'y'))
mesh

Mesh(array([[0, 1],
       [2, 3],
       [4, 5],
       [6, 7]]), ('x', 'y'))

In [45]:
input_data = np.arange(8 * 2).reshape(8, 2)
input_data

array([[ 0,  1],
       [ 2,  3],
       [ 4,  5],
       [ 6,  7],
       [ 8,  9],
       [10, 11],
       [12, 13],
       [14, 15]])

In [46]:
in_axis_resources=None
out_axis_resources=PartitionSpec('x', 'y')

In [47]:
f = pjit(
  lambda x: x,
  in_axis_resources=None,
  out_axis_resources=PartitionSpec('x', 'y'))
 
# Sends data to accelerators based on partition_spec
with maps.Mesh(mesh.devices, mesh.axis_names):
 data = f(input_data)

In [48]:
data

ShardedDeviceArray([[ 0,  1],
                    [ 2,  3],
                    [ 4,  5],
                    [ 6,  7],
                    [ 8,  9],
                    [10, 11],
                    [12, 13],
                    [14, 15]], dtype=int32)

In [49]:
data.device_buffers

[DeviceArray([[0],
              [2]], dtype=int32),
 DeviceArray([[1],
              [3]], dtype=int32),
 DeviceArray([[4],
              [6]], dtype=int32),
 DeviceArray([[5],
              [7]], dtype=int32),
 DeviceArray([[ 8],
              [10]], dtype=int32),
 DeviceArray([[ 9],
              [11]], dtype=int32),
 DeviceArray([[12],
              [14]], dtype=int32),
 DeviceArray([[13],
              [15]], dtype=int32)]

In [52]:
f = pjit(
  lambda x: x,
  in_axis_resources=None,
  out_axis_resources=PartitionSpec('x', None))
 
with maps.Mesh(mesh.devices, mesh.axis_names):
 data = f(input_data)

data

ShardedDeviceArray([[ 0,  1],
                    [ 2,  3],
                    [ 4,  5],
                    [ 6,  7],
                    [ 8,  9],
                    [10, 11],
                    [12, 13],
                    [14, 15]], dtype=int32)

In [53]:
data.device_buffers

[DeviceArray([[0, 1],
              [2, 3]], dtype=int32),
 DeviceArray([[0, 1],
              [2, 3]], dtype=int32),
 DeviceArray([[4, 5],
              [6, 7]], dtype=int32),
 DeviceArray([[4, 5],
              [6, 7]], dtype=int32),
 DeviceArray([[ 8,  9],
              [10, 11]], dtype=int32),
 DeviceArray([[ 8,  9],
              [10, 11]], dtype=int32),
 DeviceArray([[12, 13],
              [14, 15]], dtype=int32),
 DeviceArray([[12, 13],
              [14, 15]], dtype=int32)]

In [54]:
f = pjit(
  lambda x: x,
  in_axis_resources=None,
  out_axis_resources=PartitionSpec('y', None))
 
with maps.Mesh(mesh.devices, mesh.axis_names):
 data = f(input_data)

data.device_buffers

[DeviceArray([[0, 1],
              [2, 3],
              [4, 5],
              [6, 7]], dtype=int32),
 DeviceArray([[ 8,  9],
              [10, 11],
              [12, 13],
              [14, 15]], dtype=int32),
 DeviceArray([[0, 1],
              [2, 3],
              [4, 5],
              [6, 7]], dtype=int32),
 DeviceArray([[ 8,  9],
              [10, 11],
              [12, 13],
              [14, 15]], dtype=int32),
 DeviceArray([[0, 1],
              [2, 3],
              [4, 5],
              [6, 7]], dtype=int32),
 DeviceArray([[ 8,  9],
              [10, 11],
              [12, 13],
              [14, 15]], dtype=int32),
 DeviceArray([[0, 1],
              [2, 3],
              [4, 5],
              [6, 7]], dtype=int32),
 DeviceArray([[ 8,  9],
              [10, 11],
              [12, 13],
              [14, 15]], dtype=int32)]

In [55]:
f = pjit(
  lambda x: x,
  in_axis_resources=None,
  out_axis_resources=PartitionSpec(('x', 'y'), None))
 
with maps.Mesh(mesh.devices, mesh.axis_names):
 data = f(input_data)

data.device_buffers

[DeviceArray([[0, 1]], dtype=int32),
 DeviceArray([[2, 3]], dtype=int32),
 DeviceArray([[4, 5]], dtype=int32),
 DeviceArray([[6, 7]], dtype=int32),
 DeviceArray([[8, 9]], dtype=int32),
 DeviceArray([[10, 11]], dtype=int32),
 DeviceArray([[12, 13]], dtype=int32),
 DeviceArray([[14, 15]], dtype=int32)]

In [56]:
f = pjit(
  lambda x: x,
  in_axis_resources=None,
  out_axis_resources=PartitionSpec(None, 'y'))
 
with maps.Mesh(mesh.devices, mesh.axis_names):
 data = f(input_data)

data.device_buffers

[DeviceArray([[ 0],
              [ 2],
              [ 4],
              [ 6],
              [ 8],
              [10],
              [12],
              [14]], dtype=int32),
 DeviceArray([[ 1],
              [ 3],
              [ 5],
              [ 7],
              [ 9],
              [11],
              [13],
              [15]], dtype=int32),
 DeviceArray([[ 0],
              [ 2],
              [ 4],
              [ 6],
              [ 8],
              [10],
              [12],
              [14]], dtype=int32),
 DeviceArray([[ 1],
              [ 3],
              [ 5],
              [ 7],
              [ 9],
              [11],
              [13],
              [15]], dtype=int32),
 DeviceArray([[ 0],
              [ 2],
              [ 4],
              [ 6],
              [ 8],
              [10],
              [12],
              [14]], dtype=int32),
 DeviceArray([[ 1],
              [ 3],
              [ 5],
              [ 7],
              [ 9],
              [11],
     